## Undergraduate Final Project - Wind Map Modeling for West Java Region, Indonesia
### by Hafidz Rizky Firmansyah, Bandung Institute of Technology, Indonesia

This code were used to calculate the values of West Java design wind map with various Risk Categories. The result of this study were published in https://link.springer.com/article/10.1007/s13753-022-00420-7

To cite this study:

**Abdillah, M.R., Sarli, P.W., Firmansyah, H.R. et al.** Extreme Wind Variability and Wind Map Development in Western Java, Indonesia. *Int J Disaster Risk Sci* 13, 465–480 (2022).

In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import csv
import re

In [ ]:
datamatlab_windspeedori = scipy.io.loadmat('data\calibrated_era5_annmaxima.mat')
#datamatlab_windspeedori.items()

From this Matlab file, the variable to be plotted is 'gustc' , **daily wind data calibrated from ERA5 and observation data**

In [ ]:
data_wind_cal = datamatlab_windspeedori["anmax_cal"]
data_wind_raw = datamatlab_windspeedori["anmax_raw"]

In [ ]:
data_wind_raw.shape

The following, is the algorithm to find the maximum annual value of 4131 *grid* and 42 years of data from the westmost coordinate (105 E) to the eastmost coordinate (109.0 E)

After obtaining the maximum annual value for each of these coordinates, then the data is sorted from smallest to largest, then stored in the variable **df_rekap_annualmaxima_sorted**

In [ ]:
ranking = {'rank' : np.arange(1,43,1)}
yearindex = 1979
df_rekap_annualmaxima = pd.DataFrame(data = ranking)
#print(df_rekap_annualmaxima)
res = 0.05
longitude = 105.0
for lon in range(81):
    latitude = -8.0
    for lat in range(51):
        df_annualmaxima = pd.DataFrame(data = np.zeros(42))
        for yr in range(42):
            df_annualmaxima.iloc[yr,0] = data_wind_cal[lon,lat,yr]
        df_rekap_annualmaxima[str(round(longitude,3))+'BT' + str(round(latitude,3)) + 'LS']=df_annualmaxima
        latitude = latitude+res
    longitude = longitude+res
df_rekap_annualmaxima = df_rekap_annualmaxima.set_index('rank')
pd.set_option("display.precision", 3)
#print(df_rekap_annualmaxima)

df_rekap_annualmaxima_sorted = df_rekap_annualmaxima.apply(lambda x: x.sort_values().values)
df_rekap_annualmaxima_sorted.shape

In [ ]:
df_rekap_annualmaxima.iloc[:,2]

In [ ]:
def get_latlon(s):
    lonlat = re.findall('[0-9]+\.[0-9]+',s)
    return float(lonlat[0]),-float(lonlat[1])

## Graphical Method
This graphical method uses the usual *plotting* with the *polyfit* feature of NumPy to get the $\mu$ and $\sigma$ parameters to calculate the design wind speed based on the return period.

**This graphical method is used as a function to be applied to all points of interest**

In [ ]:
def hitungrpgrafis(windspeedmaxima):
    data = {'data' : np.zeros(42)}
    df_perhitungan = pd.DataFrame(data = data)
    df_perhitungan['rank'] = np.arange(1,len(df_perhitungan)+1)
    df_perhitungan['gringorten'] = (df_perhitungan['rank']-0.44)/(len(df_perhitungan)+0.12)
    df_perhitungan['y'] = -np.log(-np.log(df_perhitungan['gringorten']))
    m,b = np.polyfit(df_perhitungan['y'],windspeedmaxima,1)
    return m,b

## *Methods of Moments*
In the *Method of Moments* (MOM) method, the location and scale parameters $\mu$ and $\sigma$ can be approximated by the formula:

$$\hat{\sigma} = \frac{s\sqrt{6}}{\pi}$$
$$\hat{\mu} = \bar x - \gamma\hat{\sigma}$$

Where $\bar{x}$ and $s$ are the average and standard deviation of the maximum annual wind data

**This MoM method is used as a function to be applied to all points of interest**

In [ ]:
def hitungrpmom(windspeedmaxima):
    betamom = np.std(windspeedmaxima)*(6**0.5)/np.pi
    alphamom = np.mean(windspeedmaxima)-0.577216*betamom
    return betamom,alphamom

## *Methods of L-Moments* (LMOM)

The estimators of the value of $\beta$ and $\alpha$ in the L-Moment method are:
$$\hat{\sigma} = \frac{L_2}{ln(2)}$$
$$\hat{\mu} = b_0 - \gamma\hat{\sigma}$$

Where :
$$L_2 = 2b_1 - b_0 $$
$$b_1 = \frac{1}{n} \sum_{i=1}^{n} \frac{(i-1)}{(n-1)}x_{i:n}$$
$$b_0 = \frac{1}{n} \sum_{i=1}^{n} x_{i:n}$$

While the value of $\gamma$ is Euler's constant taken at 0.577216 and n is the number of samples, namely the number of years of calibrated observation data.

**This LMoM method is used as a function to be applied to all points of interest**

In [ ]:
from numpy import log as ln
def hitungrplmom(windspeedmaxima):
    n = len(windspeedmaxima)
    data = {'data' : np.zeros(42)}
    data['rank'] = np.arange(1,len(windspeedmaxima)+1)
    data['b1count'] = (data['rank']-1)/(n-1)*windspeedmaxima
    b1 = np.sum(data['b1count'])/n
    b0 = np.sum(windspeedmaxima)/n
    L2 = 2*b1-b0
    betalmom = L2/ln(2)
    alphalmom = b0-0.577216*betalmom
    return betalmom,alphalmom

## *Maximum Likelihood Estimation* (MLE) Method

The MLE method is a technique to find the most probable value of the $\alpha$ and $\beta$ parameter values from the sample. This method adopts parameter values by maximizing the likelihood function with the formula:

$$L(\alpha,\beta)=\prod_{i=1}^{n} f(\alpha,\beta,x_i)$$

Where $f$ has a PDF probability function of :
$$f(x) = \frac{1}{\beta} e^{-e^{\frac{-(x_i-\alpha)}{\beta}}}e^{\frac{-(x_i-\alpha)}{\beta}} ...(1) $$

With constraints :
    $$-\infty < x < \infty $$
    $$-\infty < \alpha < \infty$$
    $$ \beta > 0$$
By taking the derivative of equation (1), we get the $\alpha$ and $\beta$ value estimators using the MLE method :

$$\Large \hat{\alpha} = -\hat{\beta} \ln{[\frac{1}{n} \sum_{i=1}^{n} e^{\frac{-x_i}{\hat{\beta}}}]}$$
$$\Large \hat{\beta} = \bar{x}-\frac{\sum_{i=1}^{n} x_i e^{\frac{x_i}{\hat{\beta}}}}{\sum_{i=1}^{n} e^{\frac{x_i}{\hat{\beta}}}} $$

Where the value of $\hat{\beta}$ is searched by iterating the value of $\hat{\beta}$ until it reaches the point where $\hat{\beta_{i+1}} \approx \hat{\beta_{i}} $

**This MLE method is used as a function to be applied to all points of interest**

In [ ]:
# import time
# start_time = time.process_time()
def hitungrpmle(windspeedmaxima):
    n = len(windspeedmaxima)
    #Estimasi Parameter
    betamle = 1.39
    beta_mle_new = 1.4
    data = {'data' : np.zeros(41)}
    count = 1
    diverge = False
    #Iterasi nilai beta
    while (np.abs(betamle-beta_mle_new) > 0.001) & (count < 100):
        betamle = beta_mle_new
        data['x'] = windspeedmaxima*np.exp(-windspeedmaxima/betamle)
        data['y'] = np.exp(-windspeedmaxima/betamle)
        beta_mle_new = np.mean(windspeedmaxima)-(np.sum(data['x'])/np.sum(data['y']))
        avgbeta = (betamle+beta_mle_new)/2
        #print(beta_mle_new)
        count += 1
    if (count == 100):
        betamle_true = avgbeta
        diverge = True
    else:
        betamle_true = beta_mle_new
    #Selesai iterasi nilai beta, mencari nilai alpha
    alphamle = -betamle_true*ln(1/n*np.sum(data['y']))
    return betamle_true,alphamle
# print(time.process_time() - start_time, "seconds")

## The following is a function for testing plotting beta coefficient for MLE method during the iteration process

In [ ]:
import matplotlib.pyplot as plt
def plotrpmle(windspeedmaxima,judul):
    rekapbeta = pd.DataFrame(columns = ['iterasi','beta','betanew'],
                                index = np.arange(100))
    n = len(windspeedmaxima)
    #Estimasi Parameter
    betamle = 1.39
    beta_mle_new = 1.4
    rekapbeta.at[0,'beta'] = betamle
    rekapbeta.at[0,'betanew'] = beta_mle_new
    rekapbeta.at[:,'iterasi'] = np.arange(100)
    #rekapbeta.head()
    data = {'data' : np.zeros(41)}
    count = 1
    #Iterasi nilai beta
    while (np.abs(betamle-beta_mle_new) > 0.001) & (count < 100):
        betamle = beta_mle_new
        rekapbeta.at[count,'beta'] = betamle
        data['x'] = windspeedmaxima*np.exp(-windspeedmaxima/betamle)
        data['y'] = np.exp(-windspeedmaxima/betamle)
        beta_mle_new = np.mean(windspeedmaxima)-(np.sum(data['x'])/np.sum(data['y']))
        rekapbeta.at[count,'betanew'] = beta_mle_new
        avgbeta = (betamle+beta_mle_new)/2
        count += 1
    if (count == 100):
        betamle_true = avgbeta
    else:
        betamle_true = beta_mle_new
    #Selesai iterasi nilai beta, mencari nilai alpha
    alphamle = -betamle_true*ln(1/n*np.sum(data['y']))

    rekapbeta = rekapbeta.dropna()

    #Plotting
    plt.figure(figsize=(15,10))
    plt.plot(rekapbeta['iterasi'],rekapbeta['beta'],label = 'beta',marker = 'o')
    plt.plot(rekapbeta['iterasi'],rekapbeta['betanew'],label = 'beta new',marker = 'D')

    plt.title('Konvergensi nilai beta pada metode MLE di '+judul,
             fontsize = 15)
    plt.xlabel('Iterasi',fontsize = 14)
    plt.ylabel('Value',fontsize = 14)
    plt.legend(fontsize='large')
    plt.show()

**Beta value convergence at location 108.0E-7.7S (Location in Pamanukan)**

In [ ]:
plotrpmle(df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.0BT-7.7LS")],'Tasikmalaya')

**Beta value convergence at location 107.75E-6.5S (Location in Subang)**

In [ ]:
plotrpmle(df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.7BT-6.5LS")],'Subang')

**Beta value convergence at location 106.7E-6.2S (Location in South Jakarta)**

In [ ]:
plotrpmle(df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("106.7BT-6.2LS")],
         'Jakarta Selatan')

**Beta value convergence at location 108.75E-6.25S (Location in the Northern Sea of Cirebon)**

In [ ]:
plotrpmle(df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.7BT-6.2LS")],
         'Laut Utara Cirebon')

## Gumbel Distribution Validation in Mekarsari with Anderson-Darling test

In [ ]:
import scipy.stats
import scipy as sp
def testAD(df):
    dftinjau = df.astype(np.float64)
    dftinjau = dftinjau.to_list()
    a,b,c = sp.stats.anderson(dftinjau,dist='gumbel_r')
    stat_value = a
    crit_value = b[2]
    sig_value = c[2]
    return stat_value,crit_value,sig_value

In [ ]:
rekap = pd.DataFrame(data = np.zeros((7,2)),columns = ['AD value','critical value'],
                    index = np.arange(7))
rekap

df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.0BT-6.4LS")]
a,b,c = testAD(df)
rekap.iloc[0,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("106.9BT-6.4LS")]
a,b,c = testAD(df)
rekap.iloc[1,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.1BT-6.4LS")]
a,b,c = testAD(df)
rekap.iloc[2,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.0BT-6.5LS")]
a,b,c = testAD(df)
rekap.iloc[3,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.1BT-6.5LS")]
a,b,c = testAD(df)
rekap.iloc[4,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("106.9BT-6.5LS")]
a,b,c = testAD(df)
rekap.iloc[5,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("107.0BT-6.3LS")]
a,b,c = testAD(df)
rekap.iloc[6,:] = [a,b]
#rekap

In [ ]:
import matplotlib.pyplot as plt
x = range(7)
y = np.array(rekap.iloc[:,0])
z = np.array(rekap.iloc[:,1])
labels = ["107.0BT-6.4LS","106.9BT-6.4LS","107.1BT-6.4LS","107.0BT-6.5LS",
         "107.1BT-6.5LS","106.9BT-6.5LS","107.0BT-6.3LS"]
plt.figure(figsize=(15,5))
plt.xticks(x, labels,rotation = 25)
plt.scatter(x, y)
plt.plot(x,z)
plt.legend(['AD Value','Critical Limit Value'])
plt.title('Anderson-Darling Test Results for Grids around AWS Mekarsari Station',
          fontsize = 15)
plt.fill_between(x,0, z,color = 'green',alpha = 0.2)
plt.fill_between(x, z, 1,color='red', alpha = 0.2)
plt.savefig('AD_Mekarsari.png')
plt.show()

## Gumbel Distribution Validation in Tasikmalaya with Anderson-Darling test

In [ ]:
rekap2 = pd.DataFrame(data = np.zeros((7,2)),columns = ['AD value','critical value'],
                    index = np.arange(7))
rekap2

df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.0BT-7.3LS")]
a,b,c = testAD(df)
rekap2.iloc[0,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.0BT-7.4LS")]
a,b,c = testAD(df)
rekap2.iloc[1,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.1BT-7.3LS")]
a,b,c = testAD(df)
rekap2.iloc[2,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.1BT-7.4LS")]
a,b,c = testAD(df)
rekap2.iloc[3,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.1BT-7.5LS")]
a,b,c = testAD(df)
rekap2.iloc[4,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.2BT-7.3LS")]
a,b,c = testAD(df)
rekap2.iloc[5,:] = [a,b]
df = df_rekap_annualmaxima_sorted.iloc[:,df_rekap_annualmaxima_sorted.columns.get_loc("108.2BT-7.4LS")]
a,b,c = testAD(df)
rekap2.iloc[6,:] = [a,b]
#rekap2

In [ ]:
x = range(7)
y = np.array(rekap2.iloc[:,0])
z = np.array(rekap2.iloc[:,1])
labels = ["108.0BT-7.3LS","108.0BT-7.4LS","108.1BT-7.3LS","108.1BT-7.4LS",
         "108.1BT-7.5LS","108.2BT-7.3LS","108.2BT-7.4LS"]
plt.figure(figsize=(15,5))
plt.xticks(x, labels,rotation = 25)
plt.scatter(x, y)
plt.plot(x,z)
plt.legend(['AD Value','Critical Limit Value'])
plt.title('Anderson-Darling Test Results for Grids around AWS Tasikmalaya Station',
          fontsize = 15)
plt.fill_between(x,0, z,color = 'green',alpha = 0.2)
plt.fill_between(x, z, 1,color='red', alpha = 0.2)
plt.savefig('AD_Tasikmalaya.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df = df.to_frame()
df['rank'] = np.arange(1,len(df)+1)
df['gringorten'] = (df['rank']-0.44)/(len(df)+0.12)
df['y'] = -np.log(-np.log(df['gringorten']))

plt.figure(figsize=(15,5))
plt.scatter(df['y'],df.iloc[:,0]*3.6,color = 'red',label = 'Observasi')
m,b = np.polyfit(df['y'],df.iloc[:,0],1)
plt.title('Kecepatan Angin Desain, Metode Grafis, Koordinat 108.0BT - 7.7LS',size = 15)
plt.plot(df['y'],(m*df['y']+b)*3.6,label = 'Estimasi')
plt.xlabel('Reduced Variate')
plt.ylabel('Design Wind Speed (km/jam)')
plt.legend(fontsize='x-large')
plt.show()

# **Creating $\mu$ and $\sigma$ Parameter Matrix**

Below is an algorithm to calculate the value of $\mu$ and $\sigma$ which is calculated by various types of methods that have been created as a function above, all the values of $\mu$ and $\sigma$ that have been obtained, are stored in the **parameter variable **

There is an exception to this looping function, where when the coordinates are $(108.75 -6.5)$, $(108.75 -6.25)$ and $(109.25 -6.75)$, some rows are initially discarded because the linear regression contains data anomalies that cause the *Maximum Likelihood Estimation* function to experience infinite looping. , and because this grid does not meet the Gumbel distribution, as explained in the proof above

In [ ]:
df_rekap_annualmaxima_sorted.iloc[:,2]

In [ ]:
import time
start_time = time.process_time()
pd.set_option("display.precision", 6)
parameters = pd.DataFrame(columns = list(df_rekap_annualmaxima_sorted.columns),
                          index = ['m','b','mmom','bmom','mlmom','blmom'])
jmlkolom = len(df_rekap_annualmaxima_sorted.columns)
#print(parameters)
for i in range(jmlkolom):
    ann_maxima_tinjau = df_rekap_annualmaxima_sorted.iloc[:,i]
    b,m = hitungrpgrafis(ann_maxima_tinjau)
    parameters.iloc[0,i] = b
    parameters.iloc[1,i] = m
    betamom,alphamom = hitungrpmom(ann_maxima_tinjau)
    parameters.iloc[2,i] = betamom
    parameters.iloc[3,i] = alphamom
    betalmom,alphalmom = hitungrplmom(ann_maxima_tinjau)
    parameters.iloc[4,i] = betalmom
    parameters.iloc[5,i] = alphalmom
#     betamle,alphamle = hitungrpmle(ann_maxima_tinjau)
#     parameters.iloc[6,i] = betamle
#     parameters.iloc[7,i] = alphamle
    #print(parameters.iloc[:,i])
print(time.process_time() - start_time, "seconds")
parameters.to_csv(
    r'path',
    index = True, header = True)

In [ ]:
parameters.head(6)

In [ ]:
from sklearn.metrics import r2_score
data = {'data' : a}
df = pd.DataFrame(data = data)
df['rank'] = np.arange(1,len(df)+1)
df['gringorten'] = (df['rank']-0.44)/(len(df)+0.12)
df['y'] = -np.log(-np.log(df['gringorten']))
mg,bg = 1.5, 20
df['predict'] = mg*df['y']+bg
r2score = r2_score(df['data'],df['predict'])
print(r2score)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df['y'],df['data'])
plt.scatter(df['y'],df['predict'])

## Evaluation of the method with the smallest *error*

In [ ]:
from sklearn.metrics import r2_score

accuracy_score = pd.DataFrame(columns = list(df_rekap_annualmaxima_sorted.columns),
                          index = ['grafis','mom','lmom','most_accurate_r2','mse_g','mse_m','mse_l','most_accurate_mse'])
for i in range(81):
    for j in range(51):
        idx = i*51 + j
        a = df_rekap_annualmaxima_sorted.iloc[:,idx]
        data = {'data' : a}
        df = pd.DataFrame(data = data)
        df['rank'] = np.arange(1,len(df)+1)
        df['gringorten'] = (df['rank']-0.44)/(len(df)+0.12)
        df['y'] = -np.log(-np.log(df['gringorten']))
        
        mg,bg = parameters.iloc[0,idx], parameters.iloc[1,idx]
        df['rmse_g'] = np.sqrt((df['data'] - (mg*df['y']+bg))**2)
        df['grafis_val'] = mg*df['y']+bg
        accuracy_score.iloc[0,idx] = r2_score(df['data'],df['grafis_val'])
        accuracy_score.iloc[4,idx] = np.sum(df['rmse_g'])/np.sqrt(len(df))
        
        mm,bm = parameters.iloc[2,idx], parameters.iloc[3,idx]
        df['rmse_m'] = np.sqrt((df['data'] - (mm*df['y']+bm))**2)
        df['mom_val'] = mm*df['y']+bm
        accuracy_score.iloc[1,idx] = r2_score(df['data'],df['mom_val'])
        accuracy_score.iloc[5,idx] = np.sum(df['rmse_m'])/np.sqrt(len(df))

        ml,bl = parameters.iloc[4,idx], parameters.iloc[5,idx]
        df['rmse_l'] = np.sqrt((df['data'] - (ml*df['y']+bl))**2)
        df['lmom_val'] = ml*df['y']+bl
        accuracy_score.iloc[2,idx] = r2_score(df['data'],df['lmom_val'])
        accuracy_score.iloc[6,idx] = np.sum(df['rmse_l'])/np.sqrt(len(df))
        
        if(accuracy_score.iloc[:,idx].max() == accuracy_score.iloc[0,idx]):
            accuracy_score.iloc[3,idx] = 0
        elif(accuracy_score.iloc[:,idx].max() == accuracy_score.iloc[1,idx]):
            accuracy_score.iloc[3,idx] = 1
        elif(accuracy_score.iloc[:,idx].max() == accuracy_score.iloc[2,idx]):
            accuracy_score.iloc[3,idx] = 2

        if(accuracy_score.iloc[4:7,idx].min() == accuracy_score.iloc[4,idx]):
            accuracy_score.iloc[7,idx] = 0
        elif(accuracy_score.iloc[4:7,idx].min() == accuracy_score.iloc[5,idx]):
            accuracy_score.iloc[7,idx] = 1
        elif(accuracy_score.iloc[4:7,idx].min() == accuracy_score.iloc[6,idx]):
            accuracy_score.iloc[7,idx] = 2
#print(accuracy_score.head(8))

In [ ]:
acc_r2_matrix = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))
acc_rmse_matrix = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))

In [ ]:
pd.set_option("display.precision", 5)
for i in range(81):
    for j in range(50,-1,-1):
        acc_r2_matrix.iloc[j,i] = accuracy_score.iloc[3,(50-j)+i*51]
        
acc_r2_matrix.to_csv(
    r'path',
    index = True, header = True)

In [ ]:
pd.set_option("display.precision", 5)
for i in range(81):
    for j in range(50,-1,-1):
        acc_rmse_matrix.iloc[j,i] = accuracy_score.iloc[7,(50-j)+i*51]
        
acc_rmse_matrix.to_csv(
    r'path',
    index = True, header = True)

In [ ]:
parameters.columns.shape

In [ ]:
import matplotlib.pyplot as plt
def plotparameter(df,row1,row2,numrow):
    plt.figure(figsize=(15,6))
    plt.scatter(np.arange(row1,row2),df.iloc[0,row1:row2],color = 'red',label = 'Metode Grafis',marker = 'D')
    plt.scatter(np.arange(row1,row2),df.iloc[2,row1:row2],color = 'green',label = 'Metode MOM',marker = '.')
    plt.scatter(np.arange(row1,row2),df.iloc[4,row1:row2],color = 'blue',label = 'Metode LMOM',marker = 'o')
    plt.xlabel('Data')
    plt.ylabel('m Parameter (Gradient)')
    plt.title('Parameter gradien m di data ke '+str(row1)+'-'+str(row2),
             fontsize = 15)
    plt.legend(fontsize='large')
    plt.show()
for i in range(14):
    plotparameter(parameters,i*100,i*100+100,100)

In [ ]:
def plotparameter2(df,row1,row2,numrow):
    plt.figure(figsize=(15,7))
    plt.scatter(np.arange(row1,row2),df.iloc[1,row1:row2],color = 'red',label = 'Metode Grafis',marker = 'D')
    plt.scatter(np.arange(row1,row2),df.iloc[3,row1:row2],color = 'green',label = 'Metode MOM', marker = '^')
    plt.scatter(np.arange(row1,row2),df.iloc[5,row1:row2],color = 'blue',label = 'Metode LMOM',marker = 'o')
    plt.xlabel('Data')
    plt.ylabel('b Parameter (Intercept)')
    plt.legend(fontsize='large')
    plt.title('Parameter intercept b di data ke '+str(row1)+'-'+str(row2),
             fontsize = 15)
    plt.show()
for i in range(14):
    plotparameter2(parameters,i*100,i*100+100,100)

# Calculating Design Wind Speed with various Risk Categories
Below, is the algorithm for calculating **design wind speed** for the various return periods under review, taking into account *risk* :
- For buildings with Category I Risk, the wind with a return period for the building is 300 years
- For buildings with Category II Risk, the wind with the return period for the building is 700 years
- For buildings with Risk Category III, the wind with the return period for the building is 1700 years
- For buildings with Category IV Risk, the wind with the return period for the building is 3000 years

All of the *design wind speed* calculations presented are using the MLE *(Maximum Likelihood Estimation)* method because it gives the largest relative result compared to the other 3 methods, so it is considered more conservative

All the calculated *design wind speed* are assembled in the **designspeed** matrix

In [ ]:
import time
start_time = time.process_time()
pd.set_option("display.precision", 3)
returnperiod = [10,25,40,50,75,100,200,300,500,700,1700,3000]
nperiodeulang = len(returnperiod)
returnperiod_str = [str(i) for i in returnperiod]
returnperiod_name = ['grafis','mom','lmom']
returnperiod_index = ['metode '+str(i)+' '+str(x)+' tahun'
                      for i in returnperiod_name
                          for x in returnperiod
]

designspeed = pd.DataFrame(columns = list(df_rekap_annualmaxima_sorted.columns),
                          index = [returnperiod_index])
designspeed.insert(loc = 0,column = 'Periode Ulang', value = returnperiod*3)
designspeed.insert(loc = 1,column = 'reduced variate',
                   value = -np.log(-np.log((designspeed['Periode Ulang']-1)/(designspeed['Periode Ulang']))))
for i in range(len(df_rekap_annualmaxima_sorted.columns)):
    for j in range(nperiodeulang*3):
        if j < nperiodeulang:
            designspeed.iloc[j,i+2] = parameters.iloc[0,i]*designspeed.iloc[j,1]+parameters.iloc[1,i]
        elif j < 2*nperiodeulang:
            designspeed.iloc[j,i+2] = parameters.iloc[2,i]*designspeed.iloc[j,1]+parameters.iloc[3,i]
        elif j < 3*nperiodeulang:
            designspeed.iloc[j,i+2] = parameters.iloc[4,i]*designspeed.iloc[j,1]+parameters.iloc[5,i]
#print(designspeed.head())
designspeed.to_csv(
    r'path',
    index = True, header = True)
print(time.process_time() - start_time, "seconds")

In [ ]:
designspeed.iloc[:,13]

In [ ]:
df_rekap_annualmaxima_sorted.iloc[:,11]

In [ ]:
designspeed.shape

Below is a matrix prepared to accommodate data from the **designspeed** matrix which is a combination of 4 Risk categories, into a separate matrix for each Risk category for each *grid*. Initially this matrix is initialized with 0 (no data)

In [ ]:
#Defining Matriks untuk Plotting
plottingRisk1 = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))
plottingRisk2 = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))
plottingRisk3 = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))
plottingRisk4 = pd.DataFrame(columns = np.arange(81),
                          index = np.arange(51))
plottingRisk1.head()

In [ ]:
designspeed

In [ ]:
get_latlon('105.25BT-4.05LS')[0]

In [ ]:
#Defining Matriks untuk Plotting (2)
def toGISplot(df,n):
    plotRiskGIS = pd.DataFrame(columns = ['lat','lon','wspeed'],index=np.arange(len(df.columns)-2))
    listlon = [get_latlon(i)[0] for i in df.columns.drop(labels = ['Periode Ulang','reduced variate'])]
    listlat = [get_latlon(i)[1] for i in df.columns.drop(labels = ['Periode Ulang','reduced variate'])]
    plotRiskGIS.iloc[:,0] = listlat
    plotRiskGIS.iloc[:,1] = listlon
    plotRiskGIS.iloc[:,2] = df.iloc[n,2:].values
    return plotRiskGIS

plotRisk1GIS = toGISplot(designspeed,-5)
plotRisk2GIS = toGISplot(designspeed,-3)
plotRisk3GIS = toGISplot(designspeed,-2)
plotRisk4GIS = toGISplot(designspeed,-1)

plotRisk1GIS.to_csv(
    r'path',
    index = True, header = True)
plotRisk2GIS.to_csv(
    r'path',
    index = True, header = True)
plotRisk3GIS.to_csv(
    r'path',
    index = True, header = True)
plotRisk4GIS.to_csv(
    r'path',
    index = True, header = True)

In [ ]:
acc_rmseGIS = pd.DataFrame(columns = ['lat','lon','code'],index=np.arange(len(designspeed.columns)-2))
listlon = [get_latlon(i)[0] for i in designspeed.columns.drop(labels = ['Periode Ulang','reduced variate'])]
listlat = [get_latlon(i)[1] for i in designspeed.columns.drop(labels = ['Periode Ulang','reduced variate'])]
acc_rmseGIS.iloc[:,0] = listlat
acc_rmseGIS.iloc[:,1] = listlon
acc_rmseGIS.iloc[:,2] = accuracy_score.iloc[7,:].values
acc_rmseGIS.to_csv(
    r'path',
    index = True, header = True)

Next, we input the contents of the **designspeed** matrix into the *grid* matrix based on their respective Risk Categories to plot.

In [ ]:
#Membuat Matriks untuk Plotting 
pd.set_option("display.precision", 5)
for i in range(81):
    for j in range(50,-1,-1):
        plottingRisk1.iloc[j,i] = designspeed.iloc[-5,2+(50-j)+i*51]
        plottingRisk2.iloc[j,i] = designspeed.iloc[-3,2+(50-j)+i*51]
        plottingRisk3.iloc[j,i] = designspeed.iloc[-2,2+(50-j)+i*51]
        plottingRisk4.iloc[j,i] = designspeed.iloc[-1,2+(50-j)+i*51]

The following is an example of this plotting matrix for Risk category I, the location of the lower left is already the design wind speed for the southwest region of Java (Ujung Kulon). The upper left is already the design wind quantity (m/s) for the northwest region of Java (Serang, Tangerang, DKI Jakarta) etc.

In [ ]:
pd.set_option("display.precision", 3)
plottingRisk4

Exporting the data for this plots to MS.Excel

In [ ]:
plottingRisk1.to_csv(
    r'path',
    index = True, header = True)
plottingRisk2.to_csv(
    r'path',
    index = True, header = True)
plottingRisk3.to_csv(
    r'path',
    index = True, header = True)
plottingRisk4.to_csv(
    r'path',
    index = True, header = True)

# Comparative Plotting with Various Design Standards and Evaluating Coordinates when Maximum and Minimum Extreme Values are Obtained

In [ ]:
#Mencari lokasi data minimum dan maksimum
columnmin = designspeed.columns.get_loc('109.0BT-6.5LS')
columnmax = designspeed.columns.get_loc('106.4BT-6.9LS')
datamin = designspeed.iloc[:,columnmin]
datamax = designspeed.iloc[:,columnmax]

In [ ]:
pd.set_option("display.precision", 2)
returnperiod = [10,25,40,50,75,100,200,300,500,700,1700,3000]
jmlrp = len(returnperiod)

rekap = pd.DataFrame(returnperiod,columns = ['Periode Ulang'])
rekap['reduced variate']=-np.log(-np.log((rekap['Periode Ulang']-1)/(rekap['Periode Ulang'])))
rekap['Grafis min'] = datamin[:jmlrp].values
rekap['MOM min'] = datamin[jmlrp:2*jmlrp].values
rekap['LMOM min'] = datamin[2*jmlrp:3*jmlrp].values

rekap['Grafis max'] = datamax[:jmlrp].values
rekap['MOM max'] = datamax[jmlrp:2*jmlrp].values
rekap['LMOM max'] = datamax[2*jmlrp:3*jmlrp].values

rekap['SNI 1725-2016,low'] = 25
rekap['SNI 1725-2016,high'] = 35
rekap['PPPURG 1987,low'] = 20
rekap['PPPURG 1987,high'] = 25
rekap['HB 212-2002'] = 70-56/(rekap['Periode Ulang'])**0.1
rekap.head(12)

In [ ]:
pd.to_numeric(rekap['LMOM min'])

In [ ]:
#Import Package untuk Plotting Peta Angin
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
plt.rcParams["font.family"] = "sans-serif"

In [ ]:
#Plotting
plt.figure(figsize = (15,10))
ax = plt.subplot(111)

plt.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['LMOM min'],color = 'white')

plt.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['LMOM max'],color = 'white')

#plt.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['PPPURG 1987'],label = 'PPPURG 1987',color = 'black')
ax.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['HB 212-2002'],'k--',label = 'HB 212-2002')
ax.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['SNI 1725-2016,low'],'k-.',label = 'SNI 1725-2016, Low')
ax.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['SNI 1725-2016,high'],'k-.',label = 'SNI 1725-2016, High')
ax.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['PPPURG 1987,low'],'k.-',label = 'PPPURG 1987, Low')
ax.plot(pd.to_numeric(rekap['Periode Ulang']),rekap['PPPURG 1987,high'],'k.-',label = 'PPPURG 1987, High')

ax.fill_between(pd.to_numeric(rekap['Periode Ulang']),pd.to_numeric(rekap['LMOM min']),pd.to_numeric(rekap['LMOM max']),alpha = 0.2)

ax.text(1000, 28.6, 'Design Wind Speed, High',rotation = 7.,fontsize = 15)
ax.text(1000, 25.8, 'Design Wind Speed, Low', rotation = 4.,fontsize = 15)
ax.text(2100, 42.7, 'HB 212-2002', rotation = 5.,fontsize = 15)
ax.text(2100, 25.5, 'SNI 1725-2016, Lower Bound', fontsize = 15)
ax.text(2100, 35.5, 'SNI 1725-2016, Upper Bound', fontsize = 15)
ax.text(2100, 23.9, 'PPPURG 1987, Upper Bound', fontsize = 15)
ax.text(2100, 20.5, 'PPPURG 1987, Lower Bound', fontsize = 15)

#plt.title('Design Wind Speed Overall Result',fontsize = 20)
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xticks(fontsize= 12)
plt.yticks(fontsize= 12)
ax.set_xlabel('Return Period (year)',fontsize = 14)
ax.set_ylabel('Design Wind Speed (m/s)',fontsize = 14)
#ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=3, fancybox=True,fontsize = 14)
#labelLines(plt.gca().get_lines(),zorder=2.5)
plt.savefig('Calc vs Std.svg')
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
ax = plt.subplot(111)
ax.plot(rekap['Periode Ulang'],rekap['Grafis min'],dashes=[10, 5, 20, 5],color = 'red',label = 'Graphics low')
ax.plot(rekap['Periode Ulang'],rekap['MOM min'],'r-.',label = 'MOM low')
ax.plot(rekap['Periode Ulang'],rekap['LMOM min'],'r--',label = 'LMOM low')

ax.plot(rekap['Periode Ulang'],rekap['Grafis max'],dashes=[10, 5, 20, 5],color = 'blue',label = 'Graphics high')
ax.plot(rekap['Periode Ulang'],rekap['MOM max'],'b-.',label = 'MOM high')
ax.plot(rekap['Periode Ulang'],rekap['LMOM max'],'b--',label = 'LMOM high')

# ax.text(1000, 24.5, 'Graphics low',rotation = 8.,fontsize = 13)
# ax.text(1000, 25.1, 'MOM low',rotation = 8.,fontsize = 13)
# ax.text(1000, 25.6, 'LMOM low',rotation = 9.,fontsize = 13)
# ax.text(1000, 27.9, 'Graphics high',rotation = 12,fontsize = 13)
# ax.text(1000, 27.2, 'MOM high',rotation = 11.,fontsize = 13)
# ax.text(1000, 28.7, 'LMOM high',rotation = 11.,fontsize = 13)

#plt.title('Komparasi Perhitungan Design Wind Speed (Calibrated Data) (Koordinat Max vs Min)',fontsize = 20)
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax.set_xlabel('Return Period (year)', fontsize = 14)
ax.set_ylabel('Design Wind Speed (m/s)', fontsize = 14)
plt.xticks(fontsize=12) ; plt.yticks(fontsize=12)
ax.legend(loc='lower right', bbox_to_anchor=(1, 0),ncol=2, fancybox=True,fontsize = 16)
plt.savefig('GEV Methods.svg')
plt.show()

# Numerical Approximation for *Best Fit* Equation from existing data

It is known that from HB-212 the formula to find the design wind speed based on the return period is:

$$V_{design}=70-56R^{-0.1}$$

We will solve the equations of these various methods similar to the HB-212 equation with the form:

$$V_{design} = A - BR^{-C}$$

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def func(x,a,b,c):
    return a-b*(x**(-c))

In [ ]:
def getfunc(xdata,ydata):
    answer = curve_fit(func,xdata,ydata,bounds = (0,[70,70,1]))
    [a,b,c] = answer[0]
    return a,b,c

In [ ]:
for i in range(2,8):
    x,y,z = getfunc(np.array(rekap.iloc[:,0].values),np.array(rekap.iloc[:,i].values))
    print('%.2f %.4f %.4f'%(x,y,z))